# *Inaugural classification models.

### Based on features 'ticket_id', 'asset_id', 'root_cause', 'ticket_creation_reason','asset_type', 'latitude', 'longitude', 'tilt', 'azimuth','ticket_origin', 'date_ticket_initially_assigned', 'date_ticket_closed', 'installed_by', and 'installation_date' from Omnidian database 101, we compare K-Nearest Neighbors, Gradient Boosting, Random Forest, Bagging, and Logistic Regression.

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
import pickle

Get Data

In [3]:
data101 = pd.read_csv('data/eda101.csv')
data101.head()

,Ticket_Id,Asset_Id,Root_Cause,Ticket_Creation_Reason,asset_type,latitude,longitude,tilt,azimuth,Ticket_Origin,Date_Ticket_Initially_Assigned,Date_Ticket_Closed,installed_by,installation_date
0,23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
1,23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
2,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
3,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
4,27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,90.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00


In [4]:
data101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 14 columns):
Ticket_Id                         825 non-null int64
Asset_Id                          825 non-null int64
Root_Cause                        825 non-null object
Ticket_Creation_Reason            825 non-null object
asset_type                        825 non-null object
latitude                          825 non-null float64
longitude                         825 non-null float64
tilt                              825 non-null float64
azimuth                           825 non-null float64
Ticket_Origin                     825 non-null object
Date_Ticket_Initially_Assigned    825 non-null object
Date_Ticket_Closed                825 non-null object
installed_by                      825 non-null object
installation_date                 825 non-null object
dtypes: float64(4), int64(2), object(8)
memory usage: 90.3+ KB


Let's start by converting everything to lowercase to minimze confusion and redundancy.

In [5]:
data101.rename(str.lower, axis='columns', inplace=True)

In [6]:
data101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 14 columns):
ticket_id                         825 non-null int64
asset_id                          825 non-null int64
root_cause                        825 non-null object
ticket_creation_reason            825 non-null object
asset_type                        825 non-null object
latitude                          825 non-null float64
longitude                         825 non-null float64
tilt                              825 non-null float64
azimuth                           825 non-null float64
ticket_origin                     825 non-null object
date_ticket_initially_assigned    825 non-null object
date_ticket_closed                825 non-null object
installed_by                      825 non-null object
installation_date                 825 non-null object
dtypes: float64(4), int64(2), object(8)
memory usage: 90.3+ KB


In [7]:
data101.columns

Index(['ticket_id', 'asset_id', 'root_cause', 'ticket_creation_reason',
       'asset_type', 'latitude', 'longitude', 'tilt', 'azimuth',
       'ticket_origin', 'date_ticket_initially_assigned', 'date_ticket_closed',
       'installed_by', 'installation_date'],
      dtype='object')

Assign target to Root_Cause and Train-Test-Split

In [8]:
X = data101.drop('root_cause', axis=1).copy()
y = data101['root_cause']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [9]:
X_test.head()

,ticket_id,asset_id,ticket_creation_reason,asset_type,latitude,longitude,tilt,azimuth,ticket_origin,date_ticket_initially_assigned,date_ticket_closed,installed_by,installation_date
611,15895,101111497,communication offline,Residential Solar PV,37.818595,-121.934280,23.0,170.0,origin_omnidian_customer,2018-06-12 17:31:48,2018-08-17,Aram Solar,2012-02-10 00:00:00
174,2591,101113294,communication offline,Residential Solar PV,33.840872,-111.771766,34.0,232.0,origin_omnidian_monitoring_team,2017-08-04 22:27:05,2018-08-16,Potere Solar,2015-04-30 00:00:00
67,11278,101111915,zero production,Residential Solar PV,40.768241,-74.510319,18.0,224.0,origin_omnidian_monitoring_team,2018-03-15 18:41:26,2018-06-07,Roof Diagnostics Solar and Electric Inc,2013-09-23 00:00:00
168,16584,101113401,zero production,Residential Solar PV,33.866658,-118.387667,23.0,55.0,origin_omnidian_monitoring_team,2018-06-25 20:48:46,2018-07-26,SunStarter Solar Installations Inc,2015-11-10 00:00:00
275,11859,101113282,zero production,Residential Solar PV,34.314705,-118.432859,23.0,132.0,origin_omnidian_customer,2018-04-02 20:49:37,2018-06-15,Green Tech Solutions Inc.,2015-08-17 00:00:00


In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 611 to 527
Data columns (total 13 columns):
ticket_id                         165 non-null int64
asset_id                          165 non-null int64
ticket_creation_reason            165 non-null object
asset_type                        165 non-null object
latitude                          165 non-null float64
longitude                         165 non-null float64
tilt                              165 non-null float64
azimuth                           165 non-null float64
ticket_origin                     165 non-null object
date_ticket_initially_assigned    165 non-null object
date_ticket_closed                165 non-null object
installed_by                      165 non-null object
installation_date                 165 non-null object
dtypes: float64(4), int64(2), object(7)
memory usage: 18.0+ KB


One hot encoding: since our data is largely categorical but our models require numeric inputs, we one hot encode it.


In [11]:
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
encoder.fit(X_train)

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [12]:
X_train = pd.DataFrame(encoder.transform(X_train).toarray(),
                         columns=encoder.get_feature_names())
X_test = pd.DataFrame(encoder.transform(X_test).toarray(),
                        columns=encoder.get_feature_names())

In [13]:
# X_test = pd.DataFrame(encoder.transform(X_test).toarray(),
#                         columns=encoder.get_feature_names())

Training our models.

In [14]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [15]:
# filename = '101_knn.pkl'
# pickle.dump(knn, open(filename, 'wb'))

In [16]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
# filename = '101_lr.pkl'
# pickle.dump(lr, open(filename, 'wb'))

In [18]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=30, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [19]:
# filename = '101_gb.pkl'
# pickle.dump(gb, open(filename, 'wb'))

In [20]:
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=30)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [21]:
# filename = '101_dt.pkl'
# pickle.dump(dt, open(filename, 'wb'))

In [22]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=25565, verbose=0,
                  warm_start=False)

In [23]:
# filename = '101_bg.pkl'
# pickle.dump(bg, open(filename, 'wb'))

In [24]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train);

In [25]:
# filename = '101_rf.pkl'
# pickle.dump(rf, open(filename, 'wb'))

Let's show our results

In [26]:
knn.score(X_test, y_test)

0.5818181818181818

In [27]:
lr.score(X_test, y_test)

0.8242424242424242

In [28]:
gb.score(X_test, y_test)

0.6121212121212121

In [29]:
dt.score(X_test, y_test)

0.593939393939394

In [30]:
bg.score(X_test, y_test)

0.8363636363636363

In [31]:
rf.score(X_test, y_test)

0.8303030303030303

What may be some other classifiers?

In [27]:
# from sklearn.utils.testing import all_estimators

In [56]:
rf.predict_proba(X_test)[0]

array([0.  , 0.  , 0.  , 0.01, 0.  , 0.11, 0.  , 0.88, 0.  , 0.  ])

In [60]:
rf.predict(X_test)[0:5]

array(['root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty',
       'root_cause_normal_wear_and_tear',
       'root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty'], dtype=object)

In [94]:
l_props = lr.predict_proba(X_test)[1]

In [100]:
zipp = list(zip(lr.classes_, l_props))

In [101]:
zipp

[('root_cause_design/sale_issue', 0.00043387759236169344),
 ('root_cause_environmental', 0.004206441640046431),
 ('root_cause_homeowner', 0.014217294259570568),
 ('root_cause_installer_workmanship', 0.004648355311296379),
 ('root_cause_major_component_failure_non-warranty', 0.0036398455245150647),
 ('root_cause_major_component_failure_warranty', 0.9000584314884308),
 ('root_cause_non-service_support', 0.027058029061024055),
 ('root_cause_normal_wear_and_tear', 0.0411124777776664),
 ('root_cause_roof_issue', 0.0027033523841786256),
 ('root_cause_service_workmanship', 0.0019218949609099132)]

In [110]:
#want a dictionary that gives probability for each class, 
#prediction, and ground truth
def display_preds_truth(model, obs, X_test, y_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
#     display['prediction'] = model.predict(X_test)[obs]
    display['ground truth'] = y_test[obs]
    return display
        
    
    

In [129]:
#want a dictionary that gives probability for each class,
def display_probas(model, obs, X_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test)[obs]
    return display

In [130]:
display_probas(lr, 1, X_test)

{'root_cause_design/sale_issue': 0.00043387759236169344,
 'root_cause_environmental': 0.004206441640046431,
 'root_cause_homeowner': 0.014217294259570568,
 'root_cause_installer_workmanship': 0.004648355311296379,
 'root_cause_major_component_failure_non-warranty': 0.0036398455245150647,
 'root_cause_major_component_failure_warranty': 0.9000584314884308,
 'root_cause_non-service_support': 0.027058029061024055,
 'root_cause_normal_wear_and_tear': 0.0411124777776664,
 'root_cause_roof_issue': 0.0027033523841786256,
 'root_cause_service_workmanship': 0.0019218949609099132,
 'prediction': 'root_cause_major_component_failure_warranty'}

In [124]:
y_test

714                    root_cause_normal_wear_and_tear
605        root_cause_major_component_failure_warranty
120                    root_cause_normal_wear_and_tear
208                    root_cause_normal_wear_and_tear
380        root_cause_major_component_failure_warranty
816        root_cause_major_component_failure_warranty
575                    root_cause_normal_wear_and_tear
266        root_cause_major_component_failure_warranty
557                    root_cause_normal_wear_and_tear
825                    root_cause_normal_wear_and_tear
580                    root_cause_normal_wear_and_tear
327                    root_cause_normal_wear_and_tear
623    root_cause_major_component_failure_non-warranty
365        root_cause_major_component_failure_warranty
827                     root_cause_non-service_support
198                    root_cause_normal_wear_and_tear
456                    root_cause_normal_wear_and_tear
465                    root_cause_normal_wear_and_tear
311       